Импорт нужных библиотек

In [43]:
import pandas as pd
import numpy as np
import collections
import random
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import f1_score, mean_squared_error
from sklearn.preprocessing import OneHotEncoder

Читаем финальный датасет и собираем фичи для обучения линейной модели

In [2]:
df = pd.read_csv('teams_matches_stats-2.csv')
df.drop('Unnamed: 0', axis = 1, inplace = True)
df.sort_values(by = 'date', ignore_index = 'True', inplace = True)
data1 = df.drop([ 'Opp Formation', 'Formation', 'Captain', 'Referee','result','time','game', 'day', 'venue', 'opponent', 'season','GT', 'Gdiff'], axis = 1)

In [3]:
#Статистика для одной команды за последние 10 матчей (показатели в среднем)
def getStats(team, date, df = data1):
    srez = df[(df['team']== team)&(df['date'] < date)]
    if 0 < len(srez) < 10:
      stats = srez.drop(['team','date'], axis = 1).sum()/len(srez)
    elif len(srez) >= 10:
      stats = srez.drop(['team','date'], axis = 1)[-10:].sum()/10
    else:
      srez = df[(df['team']== team)&(df['date'] <= date)]
      stats = srez.drop(['team','date'], axis = 1).sum()

    return stats.values.tolist()

In [4]:
#Функция для сбора фичей для каждого матча
# Разница статистик двух команд за последние 10 игр в каждом матче
def GetTrain(data):
  features = []
  for i in range(len(data)):
    team1 = getStats(data['team'][i], data['date'][i])
    team2 = getStats(data['opponent'][i], data['date'][i])
    diff = [a - b for a, b in zip(team1, team2)]
    features.append(diff)
  return features

In [77]:
# Собираем фичи для каждого матча
feat_data = GetTrain(df)

In [99]:
#Собираем в датасет и добавляем колонку venue
x = pd.DataFrame(feat_data, columns = data1.drop(['team','date'], axis = 1).columns)
x = pd.concat([x, df['venue']], axis = 1)
#x = pd.get_dummies(x,prefix=['venue'], columns = ['venue'], drop_first=True)

In [100]:
for_enc = x[['venue']]

In [101]:
enc = OneHotEncoder(handle_unknown='ignore',drop = 'first', sparse_output = False)
encd = enc.fit_transform(for_enc)
one_hot_df = pd.DataFrame(encd, columns=enc.get_feature_names_out(['venue']))
x = pd.concat([x.drop(['venue'], axis = 1),one_hot_df], axis = 1)

In [102]:
#кодируем результат матча: 1 - победа интересующей команды, 0 - другой исход (ничья, проигрыш)
y = list(map((lambda i: 1 if i == 'W' else 0), df['result']))

Обучение логистической регрессии: делим на тестовую и обучающую выборку, стандартизируем обучащую и после обучения модели изучаем ее качество (f1)

In [103]:
X_train, X_test, y_train, y_test = train_test_split(x, y,
                                                    train_size=0.8,
                                                    random_state=125)

In [104]:
normalizer = StandardScaler()
scaler = normalizer.fit(X_train)
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

In [105]:
logreg = LogisticRegression()
logreg.fit(X_train_norm, y_train)

LogisticRegression()

In [106]:
y_pred = logreg.predict(X_test_norm)
y_pred

array([0, 0, 0, ..., 1, 1, 1])

In [107]:
#Предсказание вероятностей (для расчета бизнес-метрик и в дальнейшем определении ничьи)
np.set_printoptions(suppress=True)
y_pred_prob = logreg.predict_proba(X_test_norm)
y_pred_prob

array([[0.5171447 , 0.4828553 ],
       [0.54215754, 0.45784246],
       [0.58252593, 0.41747407],
       ...,
       [0.29052432, 0.70947568],
       [0.31138752, 0.68861248],
       [0.40632686, 0.59367314]])

In [108]:
#Метрика F1
f1_score(y_test, y_pred)

0.6008279124778237

Пример предсказания результата для последнего матча Барселоны

In [115]:
def GetPrediction(team1,team2,date,venue, data = df):
    teamvec1 = getStats(team1, date)
    teamvec2 = getStats(team2, date)
    diff = [[a - b] for a, b in zip(teamvec1, teamvec2)]
    diff.append([venue])
    x = pd.DataFrame(np.transpose(diff), columns = data.drop(['Opp Formation', 'Formation', 'Captain', 'Referee','result','time','game', 'day','opponent', 'season','GT', 'Gdiff','date','team'], axis = 1).columns)
    encd = enc.transform(x[['venue']])
    one_hot_df = pd.DataFrame(encd, columns=enc.get_feature_names_out(['venue']))
    x = pd.concat([x.drop(['venue'], axis = 1),one_hot_df], axis = 1)
    #x = pd.get_dummies(x,prefix=['venue'], columns = ['venue'])
    x_n = scaler.transform(x)
    y_pred = logreg.predict(x_n)
    if y_pred[0] == 1:
      return(team1+' выиграет')
    else:
      return(team1+' проиграет или ничья')

In [116]:
#сыграли в ничью (Real Betis забил в последнюю минуту :-) )
GetPrediction('Barcelona', 'Real Betis','2024-12-07', 'Home')

'Barcelona выиграет'